In [1]:
!python -V

Python 3.9.19


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/CVT--MLOps/running mlflow-examples/mlruns/1', creation_time=1725645892822, experiment_id='1', last_update_time=1725645892822, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
!pip install pyarrow  # or fastparquet


In [9]:
!pip install fastparquet

In [10]:
import pandas as pd

def read_dataframe(filename):
    # Use pd.read_parquet to read a Parquet file
    df = pd.read_parquet(filename)
    
    # Convert datetime columns to pandas datetime objects if they exist
    if 'lpep_dropoff_datetime' in df.columns:
        df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])
    if 'lpep_pickup_datetime' in df.columns:
        df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
    
    return df


In [11]:
import pandas as pd

df_train = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')


In [12]:
len(df_train), len(df_val)


(76518, 64572)

In [13]:
# Convert PULocationID and DOLocationID to strings before concatenating
df_train['PU_DO'] = df_train['PULocationID'].astype(str) + '_' + df_train['DOLocationID'].astype(str)
df_val['PU_DO'] = df_val['PULocationID'].astype(str) + '_' + df_val['DOLocationID'].astype(str)


In [14]:
# Display the first few rows of the modified DataFrames
print(df_train[['PULocationID', 'DOLocationID', 'PU_DO']].head())
print(df_val[['PULocationID', 'DOLocationID', 'PU_DO']].head())


   PULocationID  DOLocationID    PU_DO
0            43           151   43_151
1           166           239  166_239
2            41            42    41_42
3           168            75   168_75
4           265           265  265_265
   PULocationID  DOLocationID    PU_DO
0           130           205  130_205
1           152           244  152_244
2           152            48   152_48
3           152           241  152_241
4            75            42    75_42


In [15]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [16]:
df_train['duration'] = (df_train['lpep_dropoff_datetime'] - df_train['lpep_pickup_datetime']).dt.total_seconds() / 60
df_val['duration'] = (df_val['lpep_dropoff_datetime'] - df_val['lpep_pickup_datetime']).dt.total_seconds() / 60


In [17]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [18]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/opt/conda/envs/mlflow-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


np.float64(53.60353017829977)

In [19]:
import os

# Create the 'models' directory if it doesn't exist
if not os.path.exists('models'):
    os.makedirs('models')


In [20]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [21]:
with mlflow.start_run():

    mlflow.set_tag("developer", "preethi")

    mlflow.log_param("train-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

/opt/conda/envs/mlflow-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [22]:
import xgboost as xgb

In [23]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [24]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [25]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [29]:
from hyperopt import fmin, tpe, hp, Trials
from hyperopt.pyll import scope

# Define the search space with the corrected objective parameter
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), #exponential(-3), exp(0), - [0.05, 1]
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',  # Updated objective
    'seed': 42
}

# Define the objective function (example)
def objective(params):
    # Your model training and evaluation code here
    # Example:
    from xgboost import XGBRegressor
    from sklearn.datasets import load_diabetes
    from sklearn.model_selection import cross_val_score
    
    X, y = load_diabetes(return_X_y=True)
    model = XGBRegressor(**params)
    # Use cross-validation to evaluate the model
    scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
    rmse = (-scores.mean()) ** 0.5
    return rmse

# Perform hyperparameter optimization
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

print("Best Hyperparameters:", best_result)


100%|██████████| 50/50 [00:28<00:00,  1.74trial/s, best loss: 58.220117119463886]
Best Hyperparameters: {'learning_rate': np.float64(0.05050139888691769), 'max_depth': np.float64(52.0), 'min_child_weight': np.float64(19.707747714803663), 'reg_alpha': np.float64(0.007384211677294675), 'reg_lambda': np.float64(0.3332024698599833)}


In [27]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error
import mlflow
import mlflow.sklearn
import os

# Automatically log model parameters and metrics
mlflow.sklearn.autolog()

# Ensure that the directory and file exist
os.makedirs("models", exist_ok=True)  # Create the directory if it doesn't exist

# Replace this with the actual code to generate the preprocessor.b file if necessary
# For now, let's assume it should already exist. Check for its existence.
if not os.path.exists("models/preprocessor.b"):
    print("File 'models/preprocessor.b' not found. Please generate the file before running the code.")
else:
    # Loop through the models
    for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

        with mlflow.start_run():  # Start a new MLflow run

            # Log parameters
            mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
            mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

            # Log the artifact (only if it exists)
            mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

            # Initialize and train the model
            mlmodel = model_class()
            mlmodel.fit(X_train, y_train)

            # Predict and calculate RMSE
            y_pred = mlmodel.predict(X_val)
            rmse = mean_squared_error(y_val, y_pred, squared=False)

            # Log RMSE metric
            mlflow.log_metric("rmse", rmse)


File 'models/preprocessor.b' not found. Please generate the file before running the code.
